In [3]:
import psycopg2
import pandas as pd

RADSHIFT_CONN_STRING = "dbname='buzzfeed' user='radshift_reader'  host='127.0.0.1' port='5439' password='Data4me2'"

def batch_query_radshift(query):
    conn = psycopg2.connect(RADSHIFT_CONN_STRING)
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        yield row
    conn.commit()
    conn.close()

query = """
SELECT
                        buzz_id from web_app_post
                         limit 10
                                    
"""

data = [r for r in batch_query_radshift(query)]

metadata_query = """
SELECT
                        platform_data_api_post.external_id,
                        platform_data_api_post.parent_external_id,
                        platform_data_api_post.created_at,
                        platform_data_api_account.display_name,
                        platform_data_api_post_type.name,
                        platform_data_api_post.buzz_id,
                        web_app_post.uri,
                        web_app_post.name
                      FROM platform_data_api_post
                        INNER JOIN platform_data_api_account
                          ON platform_data_api_account._id = platform_data_api_post.account_id
                        INNER JOIN platform_data_api_post_type
                          ON platform_data_api_post_type._id = platform_data_api_post.post_type_id
                        inner join web_app_post
                          on platform_data_api_post.buzz_id = web_app_post.buzz_id 
                    where platform_data_api_post.created_at > '2017-01-01'
                    and ( platform_data_api_post_type.name = 'link'
                            or 
                            platform_data_api_post_type.name = 'video')
"""

data = [r for r in batch_query_radshift(metadata_query)]
df = pd.DataFrame(data)

df = df.rename(columns={0: 'external_id', 1: 'parent_external_id', 2: 'created_at', 3: 'page', 4: 'post_type', 5:'buzz_id', 6:'uri', 7: 'title' })
df.to_csv('post_metadata_6-20.csv')

ids = df['external_id']
ids = "'" + "', '".join(ids) + "'"

cols = [
    'external_id',
    'stats_ts',
  'stats_date', 
  'consumptions_by_type__link_clicks',
 'consumptions_by_type__other_clicks',
  'reactions_like_total',
  'stories_by_action_type__comment',
  'stories_by_action_type__like',
  'stories_by_action_type__share',
  'engaged_fan',
  'fan_reach',
  'impressions',
  'impressions_fan',
#   'video_complete_views_30s_organic',
#   'video_complete_views_organic'
       ]
rename = dict(zip(range(len(cols)), cols))

stats_query = """
SELECT
    external_object_id external_id,
  stats_ts,
  stats_date, 
  consumptions_by_type__link_clicks,
  consumptions_by_type__other_clicks,
  reactions_like_total,
  stories_by_action_type__comment,
  stories_by_action_type__like,
  stories_by_action_type__share,
  engaged_fan,
  fan_reach,
  impressions,
  impressions_fan
FROM
  distributed_stats_storage_facebook_post
WHERE
  external_object_id IN (%s)
""" % ids

data = [r for r in batch_query_radshift(stats_query)]
df = pd.DataFrame(data)
df = df.rename(columns=rename)
df.to_csv('post_stats_6-20.csv')

In [4]:
print(1)

1
